In [1]:
%matplotlib inline

import os
import glob

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt

import imageio
import scipy.ndimage as ndi

In [2]:
def get_data(mediafile):
    try:
        coordinates = np.load("../export/coordinates_data_resampled_without_bad_session_{}.npy".format(mediafile))
        fixations = np.load("../export/fixations_data_resampled_without_bad_session_{}.npy".format(mediafile))
        return coordinates, fixations
    except:
        print("No stored files found.. load from files first")

In [3]:
def compute_frame_saliency_map(data, display_size, mean_data=None, normalize=False):
    
    # mean data in case of multiple sessions
    if mean_data is not None:
        data = data.mean(axis=1)

    if data[0].size == 3:
        x, y, d = data.T
    else:
        x, y = data.T
        d = np.ones(data.shape[0]).T

    bins = display_size

    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins, weights=d, normed=normalize)

    heatmap = ndi.gaussian_filter(heatmap, sigma=8)

    return heatmap.T

In [4]:
def export_gaze_heatmap(mediafile):

    print("Exporting heatmap for mediafile {}".format(mediafile))
    
    coordinates, fixations = get_data(mediafile)
    
    # Compute heatmap for all frames
    display_size = (320,200)
    
    coordinates_heatmaps = []
    for frame_coord in coordinates:
        coordinates_heatmaps.append(compute_frame_saliency_map(frame_coord, display_size=display_size))

    fixations_heatmaps = []
    for frame_coord in fixations:
        fixations_heatmaps.append(compute_frame_saliency_map(frame_coord, display_size=display_size))

    coordinates_heatmaps = np.array(coordinates_heatmaps)
    fixations_heatmaps = np.array(fixations_heatmaps)

    # export heatmaps
    data = {
        "coordinates_heatmaps" : coordinates_heatmaps,
        "fixations_heatmaps" : fixations_heatmaps
    }
    np.savez_compressed("../export/gaze_heatmap_{}.npz".format(mediafile), **data)
    
    del coordinates_heatmaps
    del fixations_heatmaps

In [5]:
for mediafile in [53, 69, 90, 111]:
    export_gaze_heatmap(mediafile=mediafile)

Exporting heatmap for mediafile 53
Exporting heatmap for mediafile 69
Exporting heatmap for mediafile 90
Exporting heatmap for mediafile 111
